# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

## Обзор данных

Импортируем все необходимые для работы библиотеки. 

In [1]:
import pandas as pd 
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
import numpy as np
import scipy.stats as st

Импортируем датасеты. Посмотрим на первые строки. 

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


У нас есть следующие столбцы: 
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы); 
- product — объём запасов в скважине (тыс. баррелей) - целевой признак. 

Из условия мы знаем, что все три признака точек значимы. Более того, мы можем видеть, что они уже стандартизированы. Проверим полноту данных. Выведем информацию о датасетах. 

In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Мы видим, что данные у нас полные, пропущенных значений нет. Все необходимые столбцы представлены в правильном типе float64. Столбец `id` в обучении модели нам не пригодится. Сохраним значения этого столбца в отдельных датасетах. В остальном данные выглядят корректно, предварительной обработки не требуют. 

## Предобработка данных 

Создадим отдельные таблицы с id скважин. 

In [9]:
data_1_id = data_1['id']
data_2_id = data_2['id']
data_3_id = data_3['id']
data_1_id.head()

0    txEyH
1    2acmU
2    409Wp
3    iJLyR
4    Xdl7t
Name: id, dtype: object

In [10]:
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)
data_3 = data_3.drop(['id'], axis=1)
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4)
memory usage: 3.1 MB


Разделение датасетов на два прошло успешно. Теперь при обучении модели у нас не будет лишних признаков. 

Разделим датасеты на признаки и целевые признаки. 

In [11]:
features_1 = data_1.drop(['product'], axis=1)
target_1 = data_1['product']
features_2 = data_2.drop(['product'], axis=1)
target_2 = data_2['product']
features_3 = data_3.drop(['product'], axis=1)
target_3 = data_3['product']

features_2.head()

,f0,f1,f2
0,-15.001348,-8.276000,-0.005876
1,14.272088,-3.475083,0.999183
2,6.263187,-5.948386,5.001160
3,-13.081196,-11.506057,4.999415
4,12.702195,-8.147433,5.004363


Разделение датасетов на признаки и целевой признак прошло успешно. 

# Вывод

Мы загрузили и предобработали данные. Разделили каждый датасет на два - отдельно сохранили id скважин и признаки с целевыми признаками. После этого мы разделили данные на признаки и целевой признак. Мы не обнаружили пропусков и все данные представлены в нужном формате. Предобработка данных завершена. 

# 2. Обучение и проверка модели

## Разделение на обучающую и валидационную выборки

Разделим данные на обучающую и валидационную выборки в соотношении 75:25. 

In [12]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, 
                                                                                      target_1, 
                                                                                      test_size=0.25,
                                                                                      random_state=12)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, 
                                                                                      target_2, 
                                                                                      test_size=0.25,
                                                                                      random_state=12)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, 
                                                                                      target_3, 
                                                                                      test_size=0.25,
                                                                                      random_state=12)

print(features_train_1.shape, target_valid_1.shape)

(75000, 3) (25000,)


Разделение данных прошло успешно. 

## Обучение и проверка модели

Для решения задачи нам подходит только линейная регрессия. Обучим модель и сделаем предсказание на валидационной выборке. Напечатаем средний запас предсказанного сырья и RMSE модели. 

In [13]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predicted_1 = model.predict(features_valid_1)
mse = mean_squared_error(target_valid_1, predicted_1)
rmse = mse ** 0.5
print('Регион 1')
print('Средний запас предсказанного сырья:', predicted_1.mean())
print('RMSE модели:', rmse)

Регион 1
Средний запас предсказанного сырья: 92.13368147346337
RMSE модели: 37.78806596603057


In [14]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predicted_2 = model.predict(features_valid_2)
mse = mean_squared_error(target_valid_2, predicted_2)
rmse = mse ** 0.5
print('Регион 2')
print('Средний запас предсказанного сырья:', predicted_2.mean())
print('RMSE модели:', rmse)

Регион 2
Средний запас предсказанного сырья: 69.0378932957214
RMSE модели: 0.8885339556138777


In [15]:
model = LinearRegression()
model.fit(features_train_3, target_train_3)
predicted_3 = model.predict(features_valid_3)
mse = mean_squared_error(target_valid_3, predicted_3)
rmse = mse ** 0.5
print('Регион 3')
print('Средний запас предсказанного сырья:', predicted_3.mean())
print('RMSE модели:', rmse)

Регион 3
Средний запас предсказанного сырья: 95.05096092338655
RMSE модели: 39.800203117903365


Средний запас предсказанного сырья выше в первом и третьем регионах, при этом мы видим, что модель второго региона предсказывает данные гораздо точнее. 

# Вывод

Мы обучили модель линейной регрессии, предсказали значения на валидационных выборках, посчитали средний запас предсказанного сырья и RMSE модели. Мы заметили, что средний запас предсказанного сырья ниже во втором регионе, при этом модель там работает со значитильно более низким показателем RMSE, что указывает на наибольшую надежность предскааний. 

# 3. Подготовка к расчёту прибыли

Сохраним в отдельных переменных все ключевые значения для расчетов. Бюджет на разработку скважин в каждом регионе - 10 млрд рублей. Доход с каждой единицы продукта (указаны в тыс.баррелей) составляет 450 тыс. рублей. 

In [16]:
budget = 10000000000
product_price = 450000
repeat = 1000
need_for_region = 500

Для расчета прибыли мы будем использовать 200 лучших скважин в каждом регионе. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [17]:
need_product = budget / product_price / 200
need_product 

111.11111111111111

Достаточный объем сырья для безубыточной разработки новой скважины составляет 111 единиц продукта, что больше среднего запаса предсказанного сырья во всех трех регионах. И значительно больше среднего запаса предсказанного сырья во втором регионе. 

# Вывод

Мы сохранили в отдельных переменных все ключевые значения для расчетов. Рассчитали достаточный объем сырья для безубыточной разработки новой скважины и выяснили, что он больше среднего запаса предсказанного сырья во всех трех регионах. 

# 4. Расчёт прибыли и рисков 

Напишем функцию. Выберем 200 скважин с максимальными значениями предсказаний. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям. Рассчитаем прибыль для полученного объёма сырья.

In [18]:
def revenue(target,predictions):
    predictions = predictions.sort_values(ascending=False)
    top_200_oilers = target.loc[predictions.index][:200]
    product = top_200_oilers.sum()
    income = product * product_price
    return income - budget

Посчитаем риски и прибыль для каждого региона. 

In [19]:
state = np.random.RandomState(12)

In [20]:
values = []
for i in range(repeat):
    target_valid_1 = target_valid_1.reset_index(drop=True)
    predicted_1 = pd.Series(predicted_1)
    target_subsample_1 = target_valid_1.sample(n=need_for_region, replace=True, random_state=state)
    probs_subsample_1 = predicted_1[target_subsample_1.index]
    rev = revenue(target_subsample_1, probs_subsample_1)
    values.append(rev)
    
values = pd.Series(values)

risk = (values < 0).mean()
mean = values.mean()
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
print('Показатели для первого региона')
print('Средняя прибыль:', mean)
print('95%-й доверительный интервал:', confidence_interval)
print('Риск убытков:', risk)

Показатели для первого региона
Средняя прибыль: 475541876.2839319
95%-й доверительный интервал: (-50083021.02576184, 1008353695.8462652)
Риск убытков: 0.039


In [21]:
values = []
for i in range(repeat):
    target_valid_2 = target_valid_2.reset_index(drop=True)
    predicted_2 = pd.Series(predicted_2)
    target_subsample_2 = target_valid_2.sample(n=need_for_region, replace=True, random_state=state)
    probs_subsample_2 = predicted_2[target_subsample_2.index]
    rev = revenue(target_subsample_2, probs_subsample_2)
    values.append(rev)
    
values = pd.Series(values)

risk = (values < 0).mean()
mean = values.mean()
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
print('Показатели для второго региона')
print('Средняя прибыль:', mean)
print('95%-й доверительный интервал:', confidence_interval)
print('Риск убытков:', risk)

Показатели для второго региона
Средняя прибыль: 538007940.7223568
95%-й доверительный интервал: (133808296.18296729, 965199255.9868712)
Риск убытков: 0.003


In [22]:
values = []
for i in range(repeat):
    target_valid_3 = target_valid_3.reset_index(drop=True)
    predicted_3 = pd.Series(predicted_3)
    target_subsample_3 = target_valid_3.sample(n=need_for_region, replace=True, random_state=state)
    probs_subsample_3 = predicted_3[target_subsample_3.index]
    rev = revenue(target_subsample_3, probs_subsample_3)
    values.append(rev)
    
values = pd.Series(values)

risk = (values < 0).mean()
mean = values.mean()
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
print('Показатели для третьего региона')
print('Средняя прибыль:', mean)
print('95%-й доверительный интервал:', confidence_interval)
print('Риск убытков:', risk)

Показатели для третьего региона
Средняя прибыль: 370412660.27151287
95%-й доверительный интервал: (-152777383.1146627, 916584603.5817643)
Риск убытков: 0.09


Из данных, полученных выше, мы можем сделать вывод, что первый и третий регионы не подходят для разработки скважин, т.к. риск убытков в этих регионах составляет более 2.5%, а именно - 3.9% в первом и 9% во втором. Таким образом получается, что наиболее выгодным для разработки скважин является второй регион. В нем наименьший риск убытков и наибольшая средняя прибыль. 

# Вывод 

Первый и третий регион не подходят для разработки скважин из-за слишком высокого риска убтыков (3.9% и 9%). Для разработки скважин рекомендуем выбрать второй регион. 

# Общий вывод

Мы загрузили и предобработали данные. Мы не обнаружили пропусков и все данные представлены в нужном формате.<br><br>

Мы обучили модель линейной регрессии, предсказали значения на валидационных выборках, посчитали средний запас предсказанного сырья и RMSE модели. Мы заметили, что средний запас предсказанного сырья ниже во втором регионе, при этом модель там работает со значитильно более низким показателем RMSE, что указывает на наибольшую надежность предскааний. <br><br>

Мы сохранили в отдельных переменных все ключевые значения для расчетов. Рассчитали достаточный объем сырья для безубыточной разработки новой скважины и выяснили, что он больше среднего запаса предсказанного сырья во всех трех регионах.<br><br>
    
<font color='black'> Первый и третий регион не подходят для разработки скважин из-за слишком высокого риска убтыков (3.9% и 9%). Для разработки скважин рекомендуем выбрать второй регион со средней прибылью 538 млн рублей. 